### Setup du nootebook:

In [ ]:
CREATE DATABASE IF NOT EXISTS ML_LAB_DB;

USE DATABASE ML_LAB_DB;

CREATE SCHEMA IF NOT EXISTS ML_SCHEMA;

USE SCHEMA ML_SCHEMA;

-- create csv format
CREATE FILE FORMAT IF NOT EXISTS CSVFORMAT 
    SKIP_HEADER = 1 
    TYPE = 'CSV';

-- create external stage with the csv format to stage the diamonds dataset
CREATE STAGE IF NOT EXISTS DIAMONDS_ASSETS 
    FILE_FORMAT = CSVFORMAT 
    URL = 's3://logbrain-datasets/ml/diamonds.csv';

### 1. Data Ingestion:

Le jeu de données **diamonds** a été largement utilisé en science des données et en apprentissage automatique. Nous l’utiliserons pour démontrer les transformateurs natifs de data science de Snowflake en termes de fonctionnalités de base de données et de compatibilité avec Spark et Pandas, en utilisant des données non synthétiques et statistiquement pertinentes, bien connues de la communauté du machine learning.

### Import Libraries

In [ ]:
# Snowpark for Python
from snowflake.snowpark.types import DoubleType
import snowflake.snowpark.functions as F

### Configuration et établissement d’une connexion à Snowflake

Les notebooks établissent une session Snowpark dans le notebook. Nous utilisant un entrepôt de données (warehouse), une base de données et un schéma tout au long de ce tutoriel.

In [ ]:
# Get Snowflake Session object
session = get_active_session()
session.sql_simplifier_enabled = True

# Add a query tag to the session.
session.query_tag = {"origin":"Axel_T", 
                     "name":"Ml_diamands", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

# Current Environment Details
print('Connection Established with the following parameters:')
print('User      : {}'.format(session.get_current_user()))
print('Role      : {}'.format(session.get_current_role()))
print('Database  : {}'.format(session.get_current_database()))
print('Schema    : {}'.format(session.get_current_schema()))
print('Warehouse : {}'.format(session.get_current_warehouse()))

### Utiliser Snowpark Read DataFrame pour charger les données depuis le fichier CSV diamonds stocké dans un stage externe

Dans le début de ce notebook, nous avons placé le fichier diamonds.csv dans un stage à partir d’un bucket S3 externe. Nous pouvons maintenant le charger.

In [ ]:
# Create a Snowpark DataFrame that is configured to load data from the CSV file
# We can now infer schema from CSV files.
diamonds_df = session.read.options({"field_delimiter": ",",
                                    "field_optionally_enclosed_by": '"',
                                    "infer_schema": True,
                                    "parse_header": True}).csv("@DIAMONDS_ASSETS")

diamonds_df

In [ ]:
# Look at descriptive stats on the DataFrame
diamonds_df.describe()

In [ ]:
diamonds_df.columns

### Nettoyage des données

Tout d’abord, mettons les en-têtes en majuscules à l’aide des opérations Snowpark DataFrame afin de les standardiser avant l’écriture des colonnes dans une table Snowflake.

In [ ]:
# Force headers to uppercase
for colname in diamonds_df.columns:
    if colname == '"table"':
       new_colname = "TABLE_PCT"
    else:
        new_colname = str.upper(colname)
    diamonds_df = diamonds_df.with_column_renamed(colname, new_colname)

diamonds_df

Next, we standardize the category formatting for **CUT** using Snowpark DataFrame operations.

This way, when we write to a Snowflake table, there will be no inconsistencies in how the Snowpark DataFrame will read in the category names. Secondly, the feature transformations on categoricals will be easier to encode.

In [ ]:
def fix_values(columnn):
    return F.upper(F.regexp_replace(F.col(columnn), '[^a-zA-Z0-9]+', '_'))

for col in ["CUT"]:
    diamonds_df = diamonds_df.with_column(col, fix_values(col))

diamonds_df

Vérifier le schéma.

In [ ]:
list(diamonds_df.schema)

Enfin, convertissons les types décimaux en **DoubleType()**, car **DecimalType()** n’est pas pris en charge par Snowflake ML pour le moment.

In [ ]:
for colname in ["CARAT", "X", "Y", "Z", "DEPTH", "TABLE_PCT"]:
    diamonds_df = diamonds_df.with_column(colname, diamonds_df[colname].cast(DoubleType()))

diamonds_df

Écrire les données nettoyées dans une table Snowflake.

In [ ]:
diamonds_df.write.mode('overwrite').save_as_table('diamonds')

Maintenant, nous allons effectuer des transformations de données avec l’API de prétraitement (Preprocessing) de Snowflake ML pour l’ingénierie des variables (feature engineering).


### 2. Transformations des variables ML

Nous allons parcourir plusieurs transformations incluses dans l’API de prétraitement de Snowflake ML.
Nous construirons également un pipeline de prétraitement qui sera utilisé dans la modélisation ML.

Remarque : toutes les transformations de variables réalisées avec Snowflake ML sont des opérations distribuées, au même titre que les opérations Snowpark DataFrame.


### Import Libraries

In [ ]:

# Snowpark for Python
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType

# Snowflake ML
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.metrics.correlation import correlation

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Misc
import json
import joblib

# warning suppresion
import warnings; warnings.simplefilter('ignore')

### Charegement des doonées:
Nous allons charger les données à partir de la table snowflake **DIAMONDS**

In [ ]:
USE DATABASE ML_LAB_DB;

USE SCHEMA ML_SCHEMA;

In [ ]:
# Get Snowflake Session object
session = get_active_session()
session.sql_simplifier_enabled = True

# Add a query tag to the session.
session.query_tag = {"origin":"Axel_T", 
                     "name":"Ml_diamands", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

# Current Environment Details
print('Connection Established with the following parameters:')
print('User      : {}'.format(session.get_current_user()))
print('Role      : {}'.format(session.get_current_role()))
print('Database  : {}'.format(session.get_current_database()))
print('Schema    : {}'.format(session.get_current_schema()))
print('Warehouse : {}'.format(session.get_current_warehouse()))



In [ ]:
# First, we read in the data from a Snowflake table into a Snowpark DataFrame
# **Change this only if you named your table something else in the data ingest notebook **
diamonds_df = session.table("DIAMONDS")

diamonds_df

### Transformations des variables

Nous allons illustrer ici quelques fonctions de transformation.

Utilisons le MinMaxScaler pour normaliser la colonne CARAT.

On applique une transformation mathématique à la variable **CARAT** (qui représente le poids du diamant) afin de ramener ses valeurs dans une plage standardisée, généralement **entre 0 et 1**.

### Principe du MinMaxScaler

Le MinMaxScaler transforme chaque valeur selon la formule suivante :

[
x_{normalisé} ={x - x_{min}} / {x_{max} - x_{min}}
]

* **x** : valeur originale
* **xmin** : valeur minimale de la colonne
* **xmax** : valeur maximale de la colonne

Ainsi :

* la plus petite valeur devient **0**
* la plus grande valeur devient **1**
* les autres valeurs sont proportionnellement réparties entre 0 et 1

### Pourquoi faire cela ?

* Mettre les variables sur la **même échelle**
* Améliorer la performance de certains algorithmes de machine learning
* Éviter qu’une variable avec de grandes valeurs numériques domine les autres

En résumé, cela permet d’uniformiser l’échelle des données avant l’entraînement d’un modèle.




In [ ]:
# Normalize the CARAT column
snowml_mms = snowml.MinMaxScaler(input_cols=["CARAT"], output_cols=["CARAT_NORM"])
normalized_diamonds_df = snowml_mms.fit(diamonds_df).transform(diamonds_df)

# Reduce the number of decimals
new_col = normalized_diamonds_df.col("CARAT_NORM").cast(DecimalType(7, 6))
normalized_diamonds_df = normalized_diamonds_df.with_column("CARAT_NORM", new_col)

normalized_diamonds_df

Utilisons l’**OrdinalEncoder** pour transformer **COLOR** et **CLARITY** de variables catégorielles en valeurs numériques afin qu’elles soient plus exploitables.

L’OrdinalEncoder est une méthode de transformation qui permet de convertir des variables catégorielles (texte) en valeurs numériques.

Dans ce cas, les colonnes COLOR et CLARITY contiennent des catégories (ex. : D, E, F pour la couleur ; IF, VVS1, VS2, etc. pour la pureté). Les algorithmes de machine learning ne peuvent pas traiter directement du texte, ils nécessitent des nombres.

In [ ]:
# Encode CUT and CLARITY preserve ordinal importance
categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
}
snowml_oe = snowml.OrdinalEncoder(input_cols=["CUT", "CLARITY"], output_cols=["CUT_OE", "CLARITY_OE"], categories=categories)
ord_encoded_diamonds_df = snowml_oe.fit(normalized_diamonds_df).transform(normalized_diamonds_df)

# Show the encoding
print(snowml_oe._state_pandas)

ord_encoded_diamonds_df

Utilisons le **OneHotEncoder** pour transformer les colonnes catégorielles en colonnes numériques.

Cela est davantage à des fins d’illustration. L’utilisation de l’**OrdinalEncoder** est plus pertinente pour le jeu de données *diamonds*, puisque **CARAT**, **COLOR** et **CLARITY** suivent tous un ordre naturel de classement.

### Explication de cette étape

### 1. Pourquoi utiliser le OneHotEncoder ?

Le **OneHotEncoder** permet de transformer une variable catégorielle (texte) en plusieurs colonnes numériques binaires (0 ou 1).

Au lieu d’attribuer un numéro à chaque catégorie (comme avec l’OrdinalEncoder), il crée **une colonne par catégorie**.

---

#### Exemple

Supposons que la colonne **COLOR** contient :

D, E, F

Après One-Hot Encoding :

| COLOR_D | COLOR_E | COLOR_F |
| ------- | ------- | ------- |
| 1       | 0       | 0       |
| 0       | 1       | 0       |
| 0       | 0       | 1       |

Chaque ligne aura un 1 uniquement dans la colonne correspondant à sa catégorie.

---

### 2. Pourquoi est-ce “à des fins d’illustration” ?

Dans le dataset **diamonds**, certaines variables suivent un **ordre naturel** :

* **CARAT** → poids croissant
* **COLOR** → qualité ordonnée (D meilleure que E, etc.)
* **CLARITY** → niveau de pureté ordonné

Le OneHotEncoder **ne conserve pas la notion d’ordre**.
Toutes les catégories sont traitées comme indépendantes, sans hiérarchie.

### Résumé

* **OneHotEncoder** : transforme les catégories en colonnes binaires, sans notion d’ordre.
* **OrdinalEncoder** : transforme les catégories en nombres en respectant un ordre.
* Dans le dataset *diamonds*, l’OrdinalEncoder est plus logique car les variables ont une hiérarchie naturelle.


In [ ]:
# Encode categoricals to numeric columns
snowml_ohe = snowml.OneHotEncoder(input_cols=["CUT", "COLOR", "CLARITY"], output_cols=["CUT_OHE", "COLOR_OHE", "CLARITY_OHE"])
transformed_diamonds_df = snowml_ohe.fit(ord_encoded_diamonds_df).transform(ord_encoded_diamonds_df)

np.array(transformed_diamonds_df.columns)

In [ ]:
transformed_diamonds_df

Enfin, nous pouvons également construire un pipeline complet de prétraitement.

Cela sera utile à la fois pour les étapes d’entraînement et d’inférence du modèle ML, afin de disposer de transformations de variables standardisées.


In [ ]:
# Categorize all the features for processing
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] # To name the ordinal encoded columns
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}

In [ ]:
# Build the pipeline
preprocessing_pipeline = Pipeline(
    steps=[
            (
                "OE",
                snowml.OrdinalEncoder(
                    input_cols=CATEGORICAL_COLUMNS,
                    output_cols=CATEGORICAL_COLUMNS_OE,
                    categories=categories,
                )
            ),
            (
                "MMS",
                snowml.MinMaxScaler(
                    clip=True,
                    input_cols=NUMERICAL_COLUMNS,
                    output_cols=NUMERICAL_COLUMNS,
                )
            )
    ]
)

PIPELINE_FILE = '/tmp/preprocessing_pipeline.joblib'
joblib.dump(preprocessing_pipeline, PIPELINE_FILE) # We are just pickling it locally first

transformed_diamonds_df = preprocessing_pipeline.fit(diamonds_df).transform(diamonds_df)
transformed_diamonds_df

In [ ]:
CREATE OR REPLACE STAGE ML_HOL_ASSETS; --to store model assets

In [ ]:
# You can also save the pickled object into the stage we created earlier for deployment
session.file.put(PIPELINE_FILE, "@ML_HOL_ASSETS", overwrite=True)

### Exploration des données

Maintenant que nous avons transformé nos variables, calculons la corrélation entre chaque paire à l’aide de la fonction **correlation()** de Snowflake ML afin de mieux comprendre leurs relations.

Remarque : la fonction de corrélation de Pearson de Snowflake ML renvoie un DataFrame Pandas.

### À quoi sert l’étape de calcul de corrélation ?

Le calcul de corrélation permet de **mesurer la relation linéaire entre deux variables numériques**.

La corrélation (souvent de Pearson) produit une valeur comprise entre **-1 et 1** :

* **1** → corrélation positive forte (les variables évoluent dans le même sens)
* **-1** → corrélation négative forte (elles évoluent en sens inverse)
* **0** → absence de relation linéaire

---

### Objectifs dans un contexte Machine Learning

#### 1. Comprendre les relations entre variables

Identifier quelles variables sont liées entre elles.
Exemple : le prix d’un diamant est souvent fortement corrélé au carat.

#### 2. Détecter la multicolinéarité

Si deux variables sont très fortement corrélées entre elles, elles apportent peut-être la même information.
Cela peut :

* compliquer l’interprétation du modèle
* dégrader certains modèles (ex. : régression linéaire)

#### 3. Sélectionner les variables pertinentes

Une variable fortement corrélée à la variable cible peut être un bon prédicteur.

#### 4. Vérifier la cohérence des transformations

Après le preprocessing, on peut s’assurer que les relations attendues sont toujours présentes.

---

### Résumé

L’étape de calcul de corrélation sert à **analyser les dépendances entre variables**, améliorer la compréhension des données et préparer une sélection de variables plus pertinente avant l’entraînement du modèle.



In [ ]:
corr_diamonds_df = correlation(df=transformed_diamonds_df)
corr_diamonds_df # This is a Pandas DataFrame

In [ ]:
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_diamonds_df, dtype=bool))

# Create a heatmap with the features
plt.figure(figsize=(7, 7))
heatmap = sns.heatmap(corr_diamonds_df, mask=mask, cmap="YlGnBu", annot=True, vmin=-1, vmax=1)

Nous constatons que **CARAT** et **PRICE** sont fortement corrélés, ce qui est logique. Examinons leur relation d’un peu plus près.

Remarque : vous devrez convertir votre DataFrame Snowpark en DataFrame Pandas afin d’utiliser **matplotlib** et **seaborn**.


In [ ]:
# Set up a plot to look at CARAT and PRICE
counts = transformed_diamonds_df.to_pandas().groupby(['PRICE', 'CARAT', 'CLARITY_OE']).size().reset_index(name='Count')

fig, ax = plt.subplots(figsize=(20, 20))
plt.title('Price vs Carat', fontsize=28)
ax = sns.scatterplot(data=counts, x='CARAT', y='PRICE', size='Count', hue='CLARITY_OE', markers='o')
ax.grid(axis='y')

# The relationship is not linear - it appears exponential which makes sense given the rarity of the large diamonds
sns.move_legend(ax, "upper left")
sns.despine(left=True, bottom=True)

Maintenant, Nous allons voir comment entraîner un modèle XGBoost avec le jeu de données diamonds.

### 3. Modélisation ML

Dans ce notebook, nous allons illustrer comment entraîner un modèle **XGBoost** avec le jeu de données *diamonds* en utilisant **XGBoost open source (OSS)**.
Nous montrerons également comment effectuer l’inférence et gérer les modèles via le **Model Registry**.


### Import Libraries

In [ ]:
USE DATABASE ML_LAB_DB;

USE SCHEMA ML_SCHEMA;

In [ ]:
# Get Snowflake Session object
session = get_active_session()
session.sql_simplifier_enabled = True

# Add a query tag to the session.
session.query_tag = {"origin":"Axel_T", 
                     "name":"Ml_diamands", 
                     "version":{"major":1, "minor":0,},
                     "attributes":{"is_quickstart":1}}

# Current Environment Details
print('Connection Established with the following parameters:')
print('User      : {}'.format(session.get_current_user()))
print('Role      : {}'.format(session.get_current_role()))
print('Database  : {}'.format(session.get_current_database()))
print('Schema    : {}'.format(session.get_current_schema()))
print('Warehouse : {}'.format(session.get_current_warehouse()))



In [ ]:
# Snowpark for Python
from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F

# Snowflake ML
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier

# data science libs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor #importer une version 2.1.3
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
import shape # impoter une version 0.48.0

# misc
import json
import joblib
import cachetools

# warning suppresion
import warnings; warnings.simplefilter('ignore')

print("shap:", shap.__version__)
print("xgboost:", xgboost.__version__)

### Charger les données et le pipeline de prétraitement.

In [ ]:
# Load in the data
diamonds_df = session.table("DIAMONDS")
diamonds_df

In [ ]:
# Categorize all the features for modeling
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] # To name the ordinal encoded columns
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

LABEL_COLUMNS = ['PRICE']
OUTPUT_COLUMNS = ['PREDICTED_PRICE']

In [ ]:
# Load the preprocessing pipeline object from stage- to do this, we download the preprocessing_pipeline.joblib.gz file to the warehouse
# where our notebook is running, and then load it using joblib.
session.file.get('@ML_HOL_ASSETS/preprocessing_pipeline.joblib.gz', '/tmp')
PIPELINE_FILE = '/tmp/preprocessing_pipeline.joblib.gz'
preprocessing_pipeline = joblib.load(PIPELINE_FILE)

### Construire un modèle de régression XGBoost open source simple.

In [ ]:
# Split the data into train and test sets
diamonds_train_df, diamonds_test_df = diamonds_df.random_split(weights=[0.9, 0.1], seed=0)

In [ ]:
diamonds_train_df

In [ ]:
diamonds_test_df

In [ ]:
# Run the train and test sets through the Pipeline object we defined earlier
train_df = preprocessing_pipeline.fit(diamonds_train_df).transform(diamonds_train_df)
test_df = preprocessing_pipeline.transform(diamonds_test_df)

# Convert to pandas dataframes to use OSS XGBoost
train_pd = train_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS+LABEL_COLUMNS).to_pandas()
test_pd = test_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS+LABEL_COLUMNS).to_pandas()

In [ ]:
train_pd

In [ ]:
test_pd

In [ ]:
# Define model config
regressor = XGBRegressor()

# Split train data into X, y
y_train_pd = train_pd.PRICE
X_train_pd = train_pd.drop(columns=['PRICE'])

# Train model
regressor.fit(X_train_pd, y_train_pd)

In [ ]:
# We can now get predictions
y_test_pred = regressor.predict(test_pd.drop(columns=['PRICE']))
y_train_pred = regressor.predict(train_pd.drop(columns=['PRICE']))

In [ ]:
y_test_pred

In [ ]:
y_train_pred

Analysons les résultats à l’aide du **MAPE** de Snowflake ML.

### Explication de l’étape : analyser les résultats avec le MAPE

Le **MAPE** (Mean Absolute Percentage Error — erreur absolue moyenne en pourcentage) est une métrique utilisée pour évaluer la performance d’un **modèle de régression**.

Il mesure l’écart moyen entre les valeurs réelles et les valeurs prédites, exprimé en **pourcentage**.

---

### Que signifie le résultat ?

* **MAPE faible** → bonnes prédictions
* **MAPE élevé** → erreurs importantes

Exemple :
Un MAPE de **5 %** signifie que, en moyenne, les prédictions s’écartent de 5 % des valeurs réelles.

---

### Pourquoi utiliser le MAPE ?

1. **Interprétation intuitive** : exprimé en pourcentage
2. **Comparaison facile** entre modèles
3. Adapté aux problèmes de prédiction de valeurs positives (comme le prix des diamants)

---

### Dans ce contexte

Après avoir entraîné le modèle XGBoost pour prédire le **PRICE**, on calcule le MAPE pour mesurer :

* la précision globale du modèle
* sa capacité à prédire correctement le prix des diamants

---

### Résumé

Cette étape consiste à **évaluer la qualité du modèle** en mesurant l’erreur moyenne en pourcentage entre les prix prédits et les prix réels.



In [ ]:
mape = mean_absolute_percentage_error(y_train_pd, y_train_pred)

In [ ]:
print(f"Mean absolute percentage error: {mape * 100} %")

Maintenant, utilisons la fonction **GridSearchCV** de **scikit-learn** pour trouver les paramètres optimaux du modèle.

In [ ]:
parameters={
        "n_estimators":[100, 200, 500],
        "learning_rate":[0.1, 0.4]
}

xgb = XGBRegressor()
clf = GridSearchCV(xgb, parameters)
clf.fit(X_train_pd, y_train_pd)

### Explication du code

Ce code réalise une **recherche des meilleurs hyperparamètres** pour un modèle **XGBoost Regressor** à l’aide de **GridSearchCV**.

---

### 1. Définition des hyperparamètres à tester

```python
parameters={
    "n_estimators":[100, 200, 500],
    "learning_rate":[0.1, 0.4]
}
```

On définit une grille de combinaisons :

* **n_estimators** : nombre d’arbres (100, 200 ou 500)
* **learning_rate** : taux d’apprentissage (0.1 ou 0.4)

Cela crée **6 combinaisons possibles** (3 × 2).

---

### 2. Création du modèle

```python
xgb = XGBRegressor()
```

On initialise un modèle de régression XGBoost.

---

### 3. Recherche des meilleurs paramètres

```python
clf = GridSearchCV(xgb, parameters)
```

GridSearchCV :

* teste toutes les combinaisons définies
* évalue chaque modèle via validation croisée
* compare leurs performances

---

### 4. Entraînement

```python
clf.fit(X_train_pd, y_train_pd)
```

Le modèle est entraîné sur les données d’entraînement.
GridSearchCV :

* entraîne plusieurs modèles
* calcule leur score
* sélectionne automatiquement la meilleure combinaison d’hyperparamètres

---

### Résumé

Ce code teste plusieurs configurations de XGBoost et sélectionne automatiquement celle qui donne les meilleures performances sur les données d’entraînement.


In [ ]:
print(clf.best_estimator_)

Nous constatons que le meilleur estimateur possède les paramètres suivants : **n_estimators = 500** et **learning_rate = 0.4**.

```
XGBRegressor(base_score=None, booster=None, callbacks=None,  
             colsample_bylevel=None, colsample_bynode=None,  
             colsample_bytree=None, device=None, early_stopping_rounds=None,  
             enable_categorical=False, eval_metric=None, feature_types=None,  
             feature_weights=None, gamma=None, grow_policy=None,  
             importance_type=None, interaction_constraints=None,  
             **learning_rate=0.4**, max_bin=None, max_cat_threshold=None,  
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,  
             max_leaves=None, min_child_weight=None, missing=nan,  
             monotone_constraints=None, multi_strategy=None, **n_estimators=500**,  
             n_jobs=None, num_parallel_tree=None, ...)

```

Nous pouvons également analyser l’ensemble des résultats de la recherche par grille (grid search).


In [ ]:
# Analyze grid search results
gs_results = clf.cv_results_
n_estimators_val = []
learning_rate_val = []
for param_dict in gs_results["params"]:
    n_estimators_val.append(param_dict["n_estimators"])
    learning_rate_val.append(param_dict["learning_rate"])
mape_val = gs_results["mean_test_score"]*-1

gs_results_df = pd.DataFrame(data={
    "n_estimators":n_estimators_val,
    "learning_rate":learning_rate_val,
    "mape":mape_val})

sns.relplot(data=gs_results_df, x="learning_rate", y="mape", hue="n_estimators", kind="line")

plt.show()

Cela est cohérent avec **learning_rate = 0,4** et **n_estimators = 500**, sélectionnés comme le meilleur estimateur avec le **MAPE le plus faible**.

Maintenant, effectuons des prédictions et analysons les résultats obtenus à partir du meilleur estimateur.


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

# Predict
opt_model = clf.best_estimator_
y_train_pred = opt_model.predict(train_pd.drop(columns=['PRICE']))

mape = mean_absolute_percentage_error(y_train_pd, y_train_pred)

print(f"Mean absolute percentage error: {mape}")

Enregistrons notre modèle optimal ainsi que ses métadonnées :

In [ ]:
optimal_model = clf.best_estimator_
optimal_n_estimators = clf.best_estimator_.n_estimators
optimal_learning_rate = clf.best_estimator_.learning_rate

optimal_mape = gs_results_df.loc[(gs_results_df['n_estimators']==optimal_n_estimators) &
                                 (gs_results_df['learning_rate']==optimal_learning_rate), 'mape'].values[0]

### Gérer les modèles à l’aide du Model Registry

Avec le Model Registry de Snowflake ML, nous disposons d’un framework natif Snowflake pour le versioning et le déploiement des modèles. Cela nous permet d’enregistrer les modèles, d’étiqueter les paramètres et les métriques, de suivre les métadonnées, de créer des versions et, en fin de compte, d’exécuter des tâches d’inférence par lots dans un entrepôt Snowflake ou de déployer vers un Snowpark Container Service.

Nous allons d’abord enregistrer nos modèles.


In [ ]:
# Get sample input data to pass into the registry logging function
X = train_df.select(CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS).limit(100)

db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())

# Define model name
model_name = "DIAMONDS_PRICE_PREDICTION"

# Create a registry and log the model
native_registry = Registry(session=session, database_name=db, schema_name=schema)

# Let's first log the very first model we trained
model_ver = native_registry.log_model(
    model_name=model_name,
    version_name='V0',
    model=regressor,
    sample_input_data=X, # to provide the feature schema
    target_platforms={'WAREHOUSE'}
)

# Add evaluation metric
model_ver.set_metric(metric_name="mean_abs_pct_err", value=mape)

# Add a description
model_ver.comment = "This is the first iteration of our Diamonds Price Prediction model. It is used for demo purposes."

# Now, let's log the optimal model from GridSearchCV
model_ver2 = native_registry.log_model(
    model_name=model_name,
    version_name='V1',
    model=optimal_model,
    sample_input_data=X, # to provide the feature schema
    target_platforms={'WAREHOUSE'}
)

# Add evaluation metric
model_ver2.set_metric(metric_name="mean_abs_pct_err", value=optimal_mape)

# Add a description
model_ver2.comment = f"This is the second iteration of our Diamonds Price Prediction model \
                        where we performed hyperparameter optimization. \
                        Optimal n_estimators & learning_rate: {optimal_n_estimators}, {optimal_learning_rate}"

In [ ]:
# Let's confirm they were added
native_registry.get_model(model_name).show_versions()

Nous pouvons voir quel est le modèle par défaut lorsque nous avons plusieurs versions portant le même nom de modèle :


In [ ]:
native_registry.get_model(model_name).default.version_name

Nous pouvons maintenant utiliser le modèle optimal pour effectuer l’inférence.


In [ ]:
model_ver = native_registry.get_model(model_name).version('v1')
result_sdf2 = model_ver.run(test_df, function_name="predict")
result_sdf2.show()

Vous pouvez également exécuter l’inférence à l’aide de SQL. Pour cela, nous utiliserons une cellule SQL et appellerons la méthode **predict** du modèle en référant le nom de l’objet modèle.


In [ ]:
test_df.write.mode('overwrite').save_as_table('DIAMONDS_TEST')

In [ ]:
--- for any other version (for example V1 below):
WITH model_version_alias AS MODEL DIAMONDS_PRICE_PREDICTION VERSION v1 SELECT 
a.*, 
model_version_alias!predict(
    a.CUT_OE,
    a.COLOR_OE, 
    a.CLARITY_OE, 
    a.CARAT, 
    a.DEPTH, 
    a.TABLE_PCT, 
    a.X, 
    a.Y, 
    a.Z
)['output_feature_0'] as prediction 
from DIAMONDS_TEST a

### Explicabilité du modèle

Un autre aspect que nous pouvons examiner pour mieux comprendre les prédictions consiste à analyser les explications sur ce que le modèle considère comme le plus déterminant lors de la génération des prédictions. Pour produire ces explications, nous utiliserons la fonction d’explicabilité intégrée de Snowflake ML.

En interne, cette fonction repose sur les **valeurs de Shapley**. Lors du processus d’entraînement, les modèles de machine learning apprennent des relations entre les entrées et les sorties, et les valeurs de Shapley permettent d’attribuer la prédiction d’un modèle à ses variables d’entrée. En considérant toutes les combinaisons possibles de variables, les valeurs de Shapley mesurent la contribution marginale moyenne de chaque variable à la prédiction du modèle. Bien que cette méthode soit coûteuse en calcul, les informations obtenues sont précieuses pour l’interprétabilité et le débogage du modèle.

Calculons maintenant ces explications à partir de notre modèle optimal.


In [ ]:
mv_explanations = model_ver.run(train_df, function_name="explain")
mv_explanations

Visualisons ces explications, car il est un peu difficile d’interpréter directement les valeurs telles quelles.

In [ ]:
import shap

# Create a sample of 1000 records
test_pd = test_df.to_pandas()
test_pd_sample = test_pd.sample(n=1000, random_state = 100).reset_index(drop=True)

# Compute shapley values for each model
shap_pd = model_ver.run(test_pd_sample, function_name="explain")

Nous constatons que **CARAT** a l’impact le plus important sur les valeurs prédites (**PRICE**), suivi de la dimension **Y**, de **CLARITY** et de **COLOR**. Cela correspond à ce que nous avions observé lors de la phase d’exploration des données dans le notebook précédent, notamment lors du graphique **PRICE vs CARAT**.

Enregistrons maintenant nos données d’entraînement dans une table Snowflake afin d’illustrer comment la version SQL de cette fonction peut également être utilisée pour générer des explications des variables.


In [ ]:
train_df.write.mode('overwrite').save_as_table('DIAMONDS_TRAIN')

Nous pouvons maintenant appeler l’API SQL en invoquant le modèle ainsi que la version que nous souhaitons évaluer afin de générer ces explications.


In [ ]:
WITH mv AS MODEL "DIAMONDS_PRICE_PREDICTION" VERSION "V1"
SELECT * FROM DIAMONDS_TRAIN,
  TABLE(mv!"EXPLAIN"(
    CUT_OE,
    COLOR_OE,
    CLARITY_OE,
    CARAT,
    DEPTH,
    TABLE_PCT,
    X,
    Y,
    Z
  ));

Pour fininr, nouas allons créerune application streamlit:

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, avg, min, max
import pandas as pd
import altair as alt
import joblib
import os

# --- App Title and Description ---
st.title("Diamond Price Prediction")
st.write(
    "This application predicts the price of a diamond based on its characteristics. "
    "Use the sliders and dropdowns below to enter the diamond's features and click 'Predict Price' to see the estimated value."
)

# --- Snowflake Session ---
session = get_active_session()

# --- Database and Schema Information ---
# This assumes the user has run the setup.sql and notebooks from the repository
DB_NAME = 'ML_LAB_DB'
SCHEMA_NAME = 'ML_SCHEMA'
STAGE_NAME = 'ML_HOL_ASSETS'
# Using the raw training data table as it contains the unprocessed features
TRAINING_TABLE = 'DIAMONDS_TRAIN' 
MODEL_NAME = 'DIAMONDS_PRICE_PREDICTION'
MODEL_VERSION = 'v1'
PIPELINE_FILE_NAME = 'preprocessing_pipeline.joblib.gz'
LOCAL_PIPELINE_PATH = f'/tmp/{PIPELINE_FILE_NAME}'

# --- Load Preprocessing Pipeline ---
# This function downloads the pipeline from stage and caches it for reuse.
@st.cache_resource
def load_pipeline():
    """Downloads the preprocessing pipeline file from stage and loads it into memory."""
    try:
        session.file.get(f'@{STAGE_NAME}/{PIPELINE_FILE_NAME}', '/tmp')
        pipeline = joblib.load(LOCAL_PIPELINE_PATH)
        return pipeline
    except Exception as e:
        st.error(f"Failed to load preprocessing pipeline from stage '@{STAGE_NAME}'.")
        st.error(f"Please ensure '{PIPELINE_FILE_NAME}' exists in the stage. Error: {e}")
        return None

try:
    with st.spinner("Loading preprocessing pipeline..."):
        preprocessing_pipeline = load_pipeline()
    if preprocessing_pipeline:
        st.success("Preprocessing pipeline loaded successfully!")
    else:
        st.stop()
except Exception as e:
    st.error(f"An error occurred while loading the pipeline: {e}")
    st.stop()


# --- Feature Input from User ---
st.sidebar.header("Diamond Features")

# Define the logical order for categorical features to be used in UI and for encoding
CATEGORICAL_ORDER = {
    "CUT": ["FAIR", "GOOD", "VERY_GOOD", "PREMIUM", "IDEAL"],
    "COLOR": ["J", "I", "H", "G", "F", "E", "D"],
    "CLARITY": ["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]
}

def get_feature_ranges(table_name):
    """Gets the min and max for numerical columns and distinct values for categorical columns."""
    # User should input raw features; the pipeline will create derived features like TABLE_PCT
    numerical_features = ['CARAT', 'DEPTH', 'TABLE_PCT', 'X', 'Y', 'Z']
    categorical_features = ['CUT', 'COLOR', 'CLARITY']

    feature_ranges = {}
    df = session.table(table_name)

    # Get min/max for numerical features
    for feature in numerical_features:
        result_row = df.agg(min(col(feature)).alias("MIN_VAL"), max(col(feature)).alias("MAX_VAL")).collect()[0]
        min_val = result_row["MIN_VAL"]
        max_val = result_row["MAX_VAL"]
        feature_ranges[feature] = (float(min_val), float(max_val))

    # Get distinct values for categorical features and sort them logically
    for feature in categorical_features:
        feature_ranges[feature] = CATEGORICAL_ORDER[feature]

    return feature_ranges

try:
    with st.spinner("Loading feature ranges from Snowflake..."):
        feature_ranges = get_feature_ranges(TRAINING_TABLE)

    # Create input widgets for raw features
    carat = st.sidebar.slider("Carat", feature_ranges['CARAT'][0], feature_ranges['CARAT'][1], float(feature_ranges['CARAT'][0] + feature_ranges['CARAT'][1]) / 2)
    depth = st.sidebar.slider("Depth", feature_ranges['DEPTH'][0], feature_ranges['DEPTH'][1], float(feature_ranges['DEPTH'][0] + feature_ranges['DEPTH'][1]) / 2)
    table = st.sidebar.slider("TABLE_PCT", feature_ranges['TABLE_PCT'][0], feature_ranges['TABLE_PCT'][1], float(feature_ranges['TABLE_PCT'][0] + feature_ranges['TABLE_PCT'][1]) / 2)
    x = st.sidebar.slider("X (Length in mm)", feature_ranges['X'][0], feature_ranges['X'][1], float(feature_ranges['X'][0] + feature_ranges['X'][1]) / 2)
    y = st.sidebar.slider("Y (Width in mm)", feature_ranges['Y'][0], feature_ranges['Y'][1], float(feature_ranges['Y'][0] + feature_ranges['Y'][1]) / 2)
    z = st.sidebar.slider("Z (Depth in mm)", feature_ranges['Z'][0], feature_ranges['Z'][1], float(feature_ranges['Z'][0] + feature_ranges['Z'][1]) / 2)

    cut = st.sidebar.selectbox("Cut", feature_ranges['CUT'])
    color = st.sidebar.selectbox("Color", feature_ranges['COLOR'])
    clarity = st.sidebar.selectbox("Clarity", feature_ranges['CLARITY'])

except Exception as e:
    st.error(f"Could not load feature ranges. Make sure the table '{TRAINING_TABLE}' exists in '{DB_NAME}.{SCHEMA_NAME}'. Error: {e}")
    st.stop()


# --- Prediction ---
if st.sidebar.button("Predict Price", type="primary"):
    with st.spinner("Preprocessing input and predicting price..."):
        try:
            # --- FEATURE ENGINEERING VIA PIPELINE ---
            # 1. Create a pandas DataFrame from user inputs with the raw feature names
            raw_features_df = pd.DataFrame(
                [[carat, cut, color, clarity, depth, table, x, y, z]],
                columns=['CARAT', 'CUT', 'COLOR', 'CLARITY', 'DEPTH', 'TABLE_PCT', 'X', 'Y', 'Z']
            )

            # 2. Run the raw features through the loaded preprocessing pipeline
            processed_features_df = preprocessing_pipeline.fit(raw_features_df).transform(raw_features_df)
            
            # 3. Get the processed feature values for the SQL query
            sql_column_order = ['CUT_OE', 'COLOR_OE', 'CLARITY_OE', 'CARAT', 'DEPTH', 'TABLE_PCT', 'X', 'Y', 'Z']
            feature_values = processed_features_df[sql_column_order].iloc[0].values.tolist()
            values_str = ', '.join(map(str, feature_values))
            
            # --- PREDICTION ---
            # Construct the SQL to call the prediction model from the registry
            # The model expects features in the order output by the pipeline
            prediction_sql = f"""
                WITH model_version_alias AS MODEL {MODEL_NAME} VERSION {MODEL_VERSION}
                SELECT model_version_alias!predict(
                    t.CUT_OE, t.COLOR_OE, t.CLARITY_OE, t.CARAT, t.DEPTH, t.TABLE_PCT, t.X, t.Y, t.Z
                )['output_feature_0'] as PREDICTION
                FROM (
                    VALUES ({values_str})
                ) AS t(CUT_OE, COLOR_OE, CLARITY_OE, CARAT, DEPTH, TABLE_PCT, X, Y, Z)
            """

            result_df = session.sql(prediction_sql).collect()
            predicted_price = result_df[0]['PREDICTION']

            st.metric("Predicted Diamond Price", f"${float(predicted_price):,.2f}")

            # --- Visualizations ---
            st.subheader("Feature Comparison")

            # Get average values from the training data for visualization
            training_df_vis = session.table(TRAINING_TABLE)
            # FIX: The column name 'TABLE' is a reserved SQL keyword and must be quoted.
            avg_values = training_df_vis.select(
                avg("CARAT").alias("Avg Carat"),
                avg("DEPTH").alias("Avg Depth"),
                avg(col("TABLE_PCT")).alias("Avg Table"),
                avg("X").alias("Avg X"),
                avg("Y").alias("Avg Y"),
                avg("Z").alias("Avg Z")
            ).to_pandas()

            # Prepare data for charting using raw user inputs for interpretability
            user_input_data = {
                'Feature': ['Carat', 'Depth', 'Table_PCT', 'X', 'Y', 'Z'],
                'Value': [carat, depth, table, x, y, z],
                'Source': 'Your Input'
            }
            avg_data = {
                'Feature': ['Carat', 'Depth', 'Table_PCT', 'X', 'Y', 'Z'],
                'Value': [
                    avg_values['Avg Carat'][0],
                    avg_values['Avg Depth'][0],
                    avg_values['Avg Table'][0],
                    avg_values['Avg X'][0],
                    avg_values['Avg Y'][0],
                    avg_values['Avg Z'][0]
                ],
                'Source': 'Training Set Average'
            }

            chart_data = pd.concat([pd.DataFrame(user_input_data), pd.DataFrame(avg_data)])

            # Create the chart
            chart = alt.Chart(chart_data).mark_bar().encode(
                x=alt.X('Feature:N', sort=None),
                y=alt.Y('Value:Q'),
                color='Source:N',
                tooltip=['Feature', 'Value', 'Source']
            ).properties(
                title="Your Diamond vs. The Average Diamond"
            )

            st.altair_chart(chart, use_container_width=True)

        except Exception as e:
            st.error(f"An error occurred during prediction. Please ensure the model '{MODEL_NAME}' version '{MODEL_VERSION}' is in the Model Registry and the pipeline file is in stage '@{STAGE_NAME}'. Error: {e}")

else:
    st.info("Adjust the features in the sidebar and click 'Predict Price'.")

# --- Instructions for Setup ---
st.sidebar.markdown("---")
st.sidebar.info(
    """
    **Setup Instructions:**
    1. Run the `setup.sql` script from the [GitHub repository](https://github.com/Snowflake-Labs/sfguide-intro-to-machine-learning-with-snowflake-ml-for-python) to create the database, schema, and tables.
    2. Run through the notebooks in the repository to train the model, create the preprocessing pipeline, and deploy them.
    3. Ensure this Streamlit app is running in a Snowflake environment with access to the created database and schema.
    """
)